In [ ]:
import os
import librosa
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# --- 1. KONFIGURASI PATH ---

DATASET_PATH = "dataset"
CATEGORIES = ["cats", "dogs"] 

# --- 2. FUNGSI EKSTRAKSI FITUR (MFCC) ---
def extract_features(file_path):
    try:
        # Load audio
        audio, sample_rate = librosa.load(file_path, res_type='kaiser_fast', duration=10) 
        # Ekstraksi MFCC (40 fitur)
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        
        # Rata-rata fitur terhadap waktu
        mfccs_processed = np.mean(mfccs.T, axis=0)
        return mfccs_processed
    except Exception as e:
        print(f"Error memproses file {file_path}: {e}")
        return None

# --- 3. LOAD DATASET ---
features = []
labels = []

print("Sedang memproses dataset audio...")

for category in CATEGORIES:
    path = os.path.join(DATASET_PATH, category)
    
    # Cek folder ada atau tidak
    if not os.path.exists(path):
        print(f"PERINGATAN: Folder tidak ditemukan: {path}")
        continue
        
    class_num = CATEGORIES.index(category) # 0 untuk cats, 1 untuk dogs
    
    # Loop semua file di dalam folder kategori
    for file_name in os.listdir(path):
        if file_name.endswith('.wav'):
            file_path = os.path.join(path, file_name)
            data = extract_features(file_path)
            
            if data is not None:
                features.append(data)
                labels.append(class_num)

print(f"Total data berhasil diproses: {len(features)} file.")

if len(features) == 0:
    print("Error: Tidak ada data yang ditemukan. Cek kembali nama folder Anda.")
    exit()

# Konversi ke Numpy Array
X = np.array(features)
y = np.array(labels)

# Split Data: 80% Training, 20% Testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Data Training: {len(X_train)}, Data Testing: {len(X_test)}")

# --- 4. MEMBANGUN MODEL (ANN) ---
model = Sequential()
# Input shape 40 karena menggunakan n_mfcc=40
model.add(Dense(256, input_shape=(40,), activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
# Output 1 neuron dengan Sigmoid (0 s.d 1)
model.add(Dense(1, activation='sigmoid')) 

model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='adam')

# --- 5. TRAINING ---
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=1)

# --- 6. EVALUASI & SIMPAN ---
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Akurasi Final pada Data Test: {accuracy*100:.2f}%")

model.save('audio_classifier_model.h5')
print("Model berhasil disimpan sebagai 'audio_classifier_model.h5'")